# Pairwise Matching su blockingRulesGPT

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import euclidean
from itertools import combinations

## Calcolo della Distanza "Intra-Cluster" Media

In [2]:
def compute_mean_distance(elements):
    pairs = list(combinations(elements,2))
    # Calcola la distanza euclidea tra ciascuna coppia di vettori
    distances = [euclidean(pair[0], pair[1]) for pair in pairs]

    # Calcola la media delle distanze
    mean_distance = np.mean(distances)

    #print("Media distanza tra i vettori di embedding:", mean_distance)

    return mean_distance

## Caricamento dello schema mediato con cluster (Rule Based)
Osservazione: sia lo Schema Mediato che i BERT Vectors sono permutati con "random_state=24", poiché il Blocking sullo Schema Mediato è stato lanciato proprio dopo una sua permutazione con "random_state=24".

In [3]:
schema_mediato = pd.read_csv("./../datasets/MediatedSchemaSemicolon.csv",sep=';')
schema_mediato = schema_mediato.sample(n=81706, random_state=24)

In [4]:
bert_vectors = pd.read_csv("./../datasets/MediatedSchemaVectors.csv")
bert_vectors = bert_vectors.sample(n=81706, random_state=24)

In [8]:
dictionary = {}
with open("./../2 - Blocking/blockingRulesGPT/clustering_full_24.txt") as file:
    for line in file:
        chiave, valore = line.split(':')
        chiave = int(chiave)
        valore = int(valore[1:-1])
        dictionary[chiave] = valore

dictionary_ordinato = dict(sorted(dictionary.items(), key=lambda x: x[0]))

In [9]:
block = list(dictionary_ordinato.values())
schema_mediato['block'] = block

In [10]:
num_blocks = schema_mediato['block'].nunique()
print(num_blocks)

7504


## AgglomerativeClustering su Vettori estratti dai Record con BERT

In [11]:
stats = {}

In [12]:
for i in range(1,num_blocks+1):
    records_cluster = schema_mediato[schema_mediato['block']==i]
    if(len(records_cluster)>1):
        vectors_cluster = bert_vectors.loc[records_cluster.index].copy() #lista degli indici di riga per il cluster corrente
        distance_threshold = compute_mean_distance(vectors_cluster.values.tolist())
        model = AgglomerativeClustering(n_clusters=None,distance_threshold=0.33*distance_threshold).fit(vectors_cluster)
        stats[i] = {}
        vectors_cluster['cluster'] = model.labels_
        for j in range(0,max(model.labels_)):
            stats[i][j] = vectors_cluster[vectors_cluster['cluster']==j].index.tolist()
    else:
        stats[i] = {}
        stats[i][0] = records_cluster.index.tolist()


## Salvataggio su File

In [14]:
with open("./results_clustersChatGPT.txt", "w") as file:
    # Iterare sul dizionario e scrivere ogni coppia chiave-valore su una nuova riga
    for chiave, valore in stats.items():
        file.write(f"{chiave}: {valore}\n")